# December 09, 2025

https://adventofcode.com/2025/day/9

In [1]:
def parse_text( text ):
    return [ [int(x) for x in line.split(",")] for line in text ]

In [2]:
test_text = f'''7,1
11,1
11,7
9,7
9,5
2,5
2,3
7,3'''

test = test_text.split("\n")
test = parse_text(test)


In [3]:
fn = "../data/2025/09.txt"
with open(fn, "r") as file:
    puzz_text = file.readlines()
puzz = [ line.strip() for line in puzz_text ]
puzz = parse_text(puzz)


# Part 1

In [4]:
def part1( puzz ):
    biggest = 0
    for i in range(len(puzz)):
        for j in range(i+1, len(puzz)):
            wid = abs(puzz[i][0]-puzz[j][0]) + 1
            hei = abs(puzz[i][1]-puzz[j][1]) + 1
            area = wid*hei
            if area > biggest:
                biggest = area
    return biggest

In [5]:
part1( test )

50

In [6]:
part1( puzz )

4773451098

# Part 2

## Attempt 1
Just plotting this thing for ideas!

In [22]:
def parse_pattern( puzz ):
    h = max( [tile[1] for tile in puzz] )
    w = max( [tile[0] for tile in puzz] )
    pattern = list()
    for _ in range(h+2):
        pattern.append( ["."] * (w+2) )

    for i, red_tile in enumerate(puzz):
        pattern[ red_tile[1] ][ red_tile[0] ] = "#"
        if i < len(puzz)-1:
            next_red = puzz[i+1]
        else:
            next_red = puzz[0]

        if red_tile[0] == next_red[0]:
            # same col, iterate row-wise
            x = min(red_tile[1], next_red[1])
            y = max(red_tile[1], next_red[1])
            for t in range(x+1,y):
                if pattern[t][red_tile[0]] == ".":
                    pattern[t][red_tile[0]] = "g"
        else:
            # same line, iterate colwise
            x = min(red_tile[0], next_red[0])
            y = max(red_tile[0], next_red[0])
            for t in range(x+1,y):
                if pattern[red_tile[1]][t] == ".":
                    pattern[red_tile[1]][t] = "g"

    return pattern

def print_pattern( pattern ):
    for line in pattern:
        print("".join([tile for tile in line]))




In [23]:
print_pattern(parse_pattern(test))

.............
.......#ggg#.
.......g...g.
..#gggg#...g.
..g........g.
..#gggggg#.g.
.........g.g.
.........#g#.
.............


In [24]:
# runs out of memory
# print_pattern(parse_pattern(puzz))

## Attempt 2
Fancy geometry
1. Orient map so that we can tell outside vs inside corners. An outside corner has 1 adjacent diagonal inside the shape. An outside corner has 3 adjacent diagonals inside the shape.
2. Loop over all red (corner) tiles.
3. Consider the rectangle with each red tile that has a greater index. (Since red tiles with lower index would have been checked in an earlier loop.)
4. If the rectangle does not include any of the appropriate adjacent diagonals, it is outside the shape. (Hence, the need for inside vs outside corners.)
4. If the rectangle has a smaller area than the current best, skip checking it.
5. Otherwise, check the for outside tiles (not red or green) inside the rectangle. If any line segments from the shape perimeter cross **into** the rectangle, then the rectangle is invalidated. Line segments that overlap with the rectangle are okay. (I think).

In [16]:
def next_tile( puzz, i ):
    if i == len(puzz) - 1:
        return puzz[0]
    return puzz[i+1]

    
def prev_tile(puzz, i):
    if i == 0:
        return puzz[-1]
    return puzz[i-1]
    

def orient_pattern( puzz ):
    '''reorient puzzle so that the first move is right, last move is up, and pattern starts on an outside corner'''
    # notation: x = line in text; y = column within that line
    # find starting tile - the leftmost tile on the highest line
    # (prioritizing higher over lefter)
    si = 0
    sx, sy = puzz[si]
    for i, tile in enumerate(puzz):
        if (tile[0] < sx) or (tile[0] == sx and tile[1] < sy):
            si = i
            sx, sy = tile

    # next tile is either down or right by construction
    nt = next_tile(puzz, si)

    if nt[0] == sx:
        # tile is to the right... keep direction the same
        new_puzz = puzz[si:] + puzz[:si]
    else:
        # tile is down... reverse direction
        new_puzz = puzz[si::-1] + puzz[-1:si:-1]
    return new_puzz

In [17]:
def dir(x, y):
    'get direction from x (tile) to y (tile)'
    if x[0] == y[0]:
        # same line, move is left or right
        if x[1] < y[1]:
            return "R"
        elif x[1] > y[1]:
            return "L"
        return None
    
    if x[1] == y[1]:
        # same col, move is up or down
        if x[0] < y[0]:
            return "D"
        elif x[0] > y[0]:
            return "U"
        return None
    
    if x[0] < y[0]:
        # x on higher line: RD or LD
        if x[1] < y[1]:
            return "RD"
        else:
            return "LD"
        
    # x on lower line: RU or LU
    if x[1] < y[1]:
        return "RU"
    else:
        return "LU"
    
def area( x, y ):
    '''area of rectangle with corners at x and y'''
    return (abs(x[0]-y[0]) + 1) * (abs(x[1]-y[1])+1)

In [63]:
def line_intersects_rectangle( x0, x1, r0, r1 ):
    '''check if line from x0 to x1 intersects the rectangle with corners at r0 and r1. Assumes x0 --- x1 is orthogonal'''
    # if both x0 and x1 lie above or both lie below the rectangle, there is no intersection
    if max(x0[0], x1[0]) <= min(r0[0], r1[0]) or min(x0[0], x1[0]) >= max(r0[0], r1[0]):
        return False
    
    # if both x0 and x1 lie left of or both lie right of the rectangle, there is no intersection
    if max(x0[1], x1[1]) <= min(r0[1], r1[1]) or min(x0[1], x1[1]) >= max(r0[1], r1[1]):
        return False

    return True

In [ ]:
def best_rect( puzz, best_so_far = 0, verbose=False ):

    for i, tile in enumerate(puzz):
        # get direction before and after tile i
        pt = prev_tile(puzz, i)
        nt = next_tile(puzz, i)
        pdir = dir( tile, pt ) # dir tile to prev_tile
        ndir = dir( tile, nt ) # dir from tile to next_tile

        # canonical dir (cdir) is L/R then U/D=4
        if pdir == "U":
            is_inside_corner = ndir == "R"
            cdir = ndir + pdir
        elif pdir == "R":
            is_inside_corner = ndir == "D"
            cdir = pdir + ndir
        elif pdir == "D":
            is_inside_corner = ndir == "L"
            cdir = ndir + pdir
        else:
            is_inside_corner = ndir == "U"
            cdir = pdir + ndir

        print( f'''Tile {i} at ({tile[0]}, {tile[1]})''' )
        if verbose:
            print( f'''Was {pdir} at ({pt[0]}, {pt[1]})''' )
            print( f'''Going {ndir} to ({nt[0]}, {nt[1]})''' )
            print( f'''It is an {"INside" if is_inside_corner else "OUTside"} corner (cdir = {cdir})''' )

        for j in range(i + 1, len(puzz)):
            jtile = puzz[j]
            # Shortcut: If area is too small, don't bother validating
            hypo_area = area(tile, jtile)
            if verbose:
                print( f'''For Tile {j} at ({jtile[0]}, {jtile[1]}) area is {hypo_area}''')
            if hypo_area <= best_so_far:
                continue

            jdir = dir(tile, jtile)

            # Check one: for an inside_corner
            # here is the logic spelled out
            # if jdir not in ["R", "L", "U", "D"]:
            #     if is_inside_corner and jdir == cdir or \     # For an inside corner, cdir lies outside the shape
            #         not is_inside_corner and jdir != cdir:    # For an outside corner, cdir is the only direction inside the shape
            #         continue
            # here is simpler equivalent logic:
            if (len(jdir) > 1) and (is_inside_corner == (jdir == cdir)):
                continue

            if verbose:
                print(f'''\t...and direction {jdir} is good''')

            # Now we have to validate there are no intersections!
            rectangle_is_good = True
            for k, ktile in enumerate(puzz):
                knt = next_tile(puzz, k)
                if line_intersects_rectangle( ktile, knt, tile, jtile ):
                    rectangle_is_good = False
                    if verbose:
                        print(f'''hypo rect invalidated by line segment starting at tile {k} from ({ktile[0]}, {ktile[1]}) to ({knt[0]}, {knt[1]})''')
                    break

            if rectangle_is_good:
                # we already checked if the area is bigger than current best
                best_so_far = hypo_area
        # end inner for loop
        if verbose:
            print("\n")
    # end outer for loop

    return best_so_far
        

In [65]:
# Check orientation function
otest = orient_pattern(test)
# otest should start with the leftmost column of the highest line
# first move should be right and last move should be up
print(test)
print(otest)

# otest shouldn't change if test pattern is in opposite direction
# it should be fixed by algo
print( otest == orient_pattern(test[::-1]))

[[7, 1], [11, 1], [11, 7], [9, 7], [9, 5], [2, 5], [2, 3], [7, 3]]
[[2, 3], [2, 5], [9, 5], [9, 7], [11, 7], [11, 1], [7, 1], [7, 3]]
True


In [66]:
best_rect( otest, verbose=True )

Tile 0 at (2, 3)
Was D at (7, 3)
Going R to (2, 5)
It is an OUTside corner (cdir = RD)
For Tile 1 at (2, 5) area is 3
	...and direction R is good
For Tile 2 at (9, 5) area is 24
	...and direction RD is good
For Tile 3 at (9, 7) area is 40
	...and direction RD is good
hypo rect invalidated by line segment starting at tile 1 from (2, 5) to (9, 5)
For Tile 4 at (11, 7) area is 50
	...and direction RD is good
hypo rect invalidated by line segment starting at tile 1 from (2, 5) to (9, 5)
For Tile 5 at (11, 1) area is 30
For Tile 6 at (7, 1) area is 18
For Tile 7 at (7, 3) area is 6


Tile 1 at (2, 5)
Was L at (2, 3)
Going D to (9, 5)
It is an OUTside corner (cdir = LD)
For Tile 2 at (9, 5) area is 8
For Tile 3 at (9, 7) area is 24
For Tile 4 at (11, 7) area is 30
For Tile 5 at (11, 1) area is 50
	...and direction LD is good
hypo rect invalidated by line segment starting at tile 6 from (7, 1) to (7, 3)
For Tile 6 at (7, 1) area is 30
	...and direction LD is good
hypo rect invalidated by line

24

In [68]:
opuzz = orient_pattern(puzz)
best_rect(opuzz)

1429075575

In [69]:
4773451098 > 1429075575

True